### Tasks

- Count how many books have been published since January 1, 2000;
- For each book, count the number of reviews and the average grade;
- Identify the publisher that has published the largest number of books thicker than 50 pages - this is how you exclude the brochure from the analysis;
- Identify the author with the highest average book rating - only count books with 50 or more ratings;
- Calculate the average number of reviews from users who gave more than 50 ratings.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
books = ''' 
SELECT *
        FROM books
        LIMIT 5
    '''
pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
authors = ''' 
SELECT *
        FROM authors
        LIMIT 5
    '''
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
publishers = ''' 
SELECT *
        FROM publishers
        LIMIT 5
    '''
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
 ratings = ''' 
SELECT *
        FROM ratings
        LIMIT 5
    '''
pd.io.sql.read_sql(ratings, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
reviews = ''' 
SELECT *
        FROM reviews
        LIMIT 5
    '''
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### How many books have been published since January 1, 2000?

In [8]:
query_1 = '''
SELECT 
COUNT (book_id) AS book_cnt
FROM books
WHERE publication_date > '2000-01-01'
'''
query_1 = pd.io.sql.read_sql(query_1, con = engine)
query_1

,book_cnt
0,819


### For each book, count the number of reviews and the average grade

In [9]:
query_2 =  '''
SELECT
    books.book_id,
    books.title,
    COUNT(DISTINCT review_id) as review_cnt,
    AVG(rating) as avg_rating
FROM 
    books
INNER JOIN reviews ON  reviews.book_id = books.book_id
INNER JOIN ratings ON  ratings.book_id = books.book_id
GROUP BY
    books.book_id
ORDER BY
    review_cnt DESC
LIMIT 10
'''
query_2 = pd.io.sql.read_sql(query_2, con = engine)
query_2

,book_id,title,review_cnt,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,696,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,207,Eat Pray Love,6,3.395833
3,627,The Alchemist,6,3.789474
4,673,The Catcher in the Rye,6,3.825581
5,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,497,Outlander (Outlander #1),6,4.125000
9,656,The Book Thief,6,4.264151


### Identify the publisher that has published the largest number of books thicker than 50 pages - this is how you exclude the brochure from the analysis

In [10]:
query_3 =  '''
SELECT
    publisher,
    COUNT(book_id) as book_cnt
FROM
    publishers
LEFT JOIN books ON  books.publisher_id = publishers.publisher_id 
WHERE 
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    book_cnt DESC
            LIMIT 1
'''
query_3 = pd.io.sql.read_sql(query_3, con = engine)
query_3

,publisher,book_cnt
0,Penguin Books,42


### Identify the author with the highest average book rating - only count books with 50 or more ratings

In [11]:
query_4 =  '''
SELECT
    authors.author,
    AVG(filtered.avg_rating) AS avg_rating
    FROM
        (SELECT
            book_id AS book_id,
            COUNT(rating_id) AS rating_cnt,
            AVG(rating) AS avg_rating
        FROM
            ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50) 
        AS filtered
    INNER JOIN books ON books.book_id = filtered.book_id
    INNER JOIN authors ON authors.author_id = books.author_id
    GROUP BY authors.author
    ORDER BY avg_rating DESC

'''
query_4 = pd.io.sql.read_sql(query_4, con = engine)
query_4

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


### Identify the author with the highest average book rating - only count books with 50 or more ratings

In [12]:
query_5 =  '''
SELECT
        AVG(subtable.count)
        FROM
        (SELECT COUNT(*)
            FROM reviews
            WHERE username IN
                (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating) > 50)
            GROUP BY username)
            AS subtable; 
'''
query_5 = pd.io.sql.read_sql(query_5, con = engine)
query_5

,avg
0,24.333333


## Conclusion: 


Since January 1, 2000, 819 books have been published. The largest number of books (books with more than 50 pages) were published by Penguin Books (42 books). J.K. Rowling is the highest rated author with an average of 4.3. Among users, he gave more than 50 ratings, the average number of reviews is 24.